### predictions

### import modules and configure notebook

In [1]:
import pandas as pd
import numpy as np
import swifter
import seaborn as sns
import matplotlib.pyplot
import pickle

from functions.model_data import bestHyperparamaters 
from functions.postproccessing import process_results

pd.set_option('max.rows', None)
pd.set_option('max.columns', None)
%matplotlib inline

### Load variables stored by data_preproccessing notebook

In [2]:
%store -r train_data_formodel
%store -r test_data
%store -r my_data
%store -r uniques
%store -r best_feats
%store -r X_test_labeled_df

In [3]:
X_test_labeled_df.head()

,Analysis,Zr90,Ba137,Sr88,Ge72,Cr52,S33,U238,Al27,B11,Mg24,Nd146,Sc45,K39,Pr141,Li7,inlierLabel
0,005_AB_1,0.24,1.06,3.563843,0.80,6.31,487.95,0.210000,306.53,58.850000,25.138079,0.17,1.00,217.05,0.04,19.35,-1
1,006_AB_1,0.30,0.58,1.180000,0.64,4.78,486.15,0.510000,413.97,85.030000,41.100000,0.17,1.25,353.39,0.06,2.61,-1
2,007_AB_1,0.25,0.49,1.250000,0.67,4.62,480.47,0.242569,364.50,80.040000,44.560000,0.24,1.12,329.78,0.07,2.29,1
3,008_AB_2,0.24,4.74,4.350000,0.71,4.46,482.84,0.030000,139.26,70.736204,10.690000,0.63,1.46,90.89,0.20,6.86,-1
4,009_AB_2,0.30,4.22,3.850000,0.98,4.65,506.40,0.060000,180.19,70.736204,18.670000,0.57,1.47,122.66,0.18,8.45,1


In [4]:
train_data_formodel.head()

,Analysis,Geology,Province,Region,Site,SubSite,Formation,Band,Nodule,Li7,Be9,B11,Mg24,Al27,Si28,P31,S33,K39,Ca42,Sc45,Ti47,V51,Cr52,Mn55,Fe56,Co59,Ni60,Cu63,Zn68,Ga69,Ge72,As75,Rb85,Sr88,Y89,Zr90,Nb93,Mo95,Cd111,In115,Sn118,Cs133,Ba137,La139,Ce140,Pr141,Nd146,Sm147,Eu153,Gd157,Tb159,Dy163,Ho165,Er166,Tm169,Yb172,Lu175,Hf178,Ta181,Pb208,Th232,U238,class
0,10_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,15.63,0.12,48.36,154.630000,943.71,464944.18,50.280000,538.57,455.94,712.39,0.42,15.58,0.27,3.30,0.69,8.46,0.05,0.80,1.62,10.82,0.25,1.22,0.16,0.43,12.94,0.88,1.51,0.09,0.05,0.02,0.00,0.05,0.01,6.54,0.84,0.95,0.23,0.87,0.16,0.04,0.16,0.02,0.11,0.03,0.06,0.01,0.02,0.00,0.04,0.01,0.24,0.07,0.05,0
1,11_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,11.50,0.09,44.77,43.735431,1077.11,465010.94,70.910000,438.20,387.82,515.24,0.44,18.47,0.29,3.45,1.01,11.59,0.11,0.36,0.53,8.93,0.34,0.85,0.10,0.45,13.22,0.95,1.74,0.07,0.01,0.02,0.00,0.04,0.02,8.04,0.92,1.01,0.23,0.98,0.18,0.04,0.18,0.02,0.13,0.03,0.06,0.01,0.04,0.01,0.05,0.00,0.07,0.08,0.04,0
2,12_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,20.05,0.06,44.88,42.700000,620.21,465295.41,104.470000,372.66,363.71,957.89,0.76,19.89,0.55,3.25,1.21,87.99,0.21,1.68,1.53,11.98,0.25,1.71,0.13,0.43,8.52,0.87,0.93,0.10,0.02,0.02,0.00,0.05,0.01,3.13,0.90,1.08,0.26,0.84,0.15,0.04,0.19,0.02,0.14,0.02,0.07,0.01,0.06,0.00,0.02,0.01,0.46,0.05,0.05,0
3,13_FH1_1_2,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_2,11.16,0.73,47.06,162.420000,1143.19,402596.61,3735.154885,1075.89,547.55,2174.30,0.43,42.30,0.67,152.42,4.84,145.34,0.30,2.45,5.02,17.15,0.35,2.13,0.84,0.76,13.16,0.97,2.00,0.10,0.29,0.18,0.01,0.78,0.04,8.74,0.93,0.95,0.21,0.75,0.13,0.04,0.25,0.02,0.09,0.03,0.05,0.00,0.03,0.00,0.08,0.00,0.64,0.05,0.03,0
4,14_FH1_1_2,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_2,17.71,0.32,48.26,33.520000,547.22,465027.11,44.440000,464.78,278.25,1551.63,0.71,11.18,0.27,2.56,1.73,25.38,0.05,0.80,0.55,9.80,0.41,1.41,0.12,0.28,9.90,0.90,0.90,0.08,0.04,0.10,0.00,0.09,0.01,2.74,0.97,1.09,0.27,1.00,0.17,0.04,0.19,0.02,0.15,0.03,0.05,0.01,0.05,0.01,0.02,0.01,0.59,0.06,0.09,0


### configurations

* scale -> True|False if set to True then features scaled to all have mean value 0 and standard deviation 1

In [5]:
save_predictions = True

In [6]:
from config_and_dependencies.config import *

### counts of instances in all classes before oversampling

In [7]:
train_data_formodel['class'].value_counts()

4     105
15    100
16     61
0      53
11     45
13     36
14     36
2      36
6      30
7      30
10     30
5      27
8      27
1      24
12     21
3      18
9      17
Name: class, dtype: int64

### The class column is stored as the variable y and the variables identified as best by the 2 feature_selection notebook are used as features

In [8]:
y = np.array(train_data_formodel[target])
X = np.array(np.array(train_data_formodel[best_feats]))

### the dimensions of the class and features are checked

In [9]:
print(X.shape)
print(y.shape)

(696, 15)
(696,)


### grid search is utilised to identify optimum hyperparamaters

In [10]:
my_optimiser = bestHyperparamaters(X, y)
my_optimiser.get_best_model()

C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
C:\Users\tom\Anaconda3\lib\site-packages\sklearn\neighbors\base.py:442: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')
C:\Users\tom\Anaconda3\lib\site-packages\sklearn\neighbors\base.py:442: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')
C:\Users\tom\Anaconda3\lib\site-packages\sklearn\neighbors\base.py:442: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')
C:\Users\tom\Anaconda3\lib\site-packages\sklearn\neighbors\base.py:442:

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] max_depth=8, min_samples_leaf=1, min_samples_split=2, n_estimators=1600 


C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


[CV]  max_depth=8, min_samples_leaf=1, min_samples_split=2, n_estimators=1600, score=0.9523810053422013, total=   4.4s
[CV] max_depth=8, min_samples_leaf=1, min_samples_split=2, n_estimators=1600 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.8s remaining:    0.0s
C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


...

C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


[CV]  max_depth=16, min_samples_leaf=10, min_samples_split=10, n_estimators=2400, score=0.9682104151090838, total=   6.2s
[CV] max_depth=16, min_samples_leaf=10, min_samples_split=10, n_estimators=2400 


C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


[CV]  max_depth=16, min_samples_leaf=10, min_samples_split=10, n_estimators=2400, score=0.9531481793841204, total=   6.2s


[Parallel(n_jobs=1)]: Done 540 out of 540 | elapsed: 55.7min finished
C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


In [11]:
best_model = my_optimiser.best_model

### Model is built for predicting source of artefacts 

In [12]:
best_model.fit(X, y)

C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=16, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2400, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

# sanity check to see model correctly predicts geological samples

In [13]:
train_check = best_model.predict(X)
sum(train_check == y)/len(y)

C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


1.0

In [14]:
train_check == y

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [15]:
train_check

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4

In [16]:
X.shape

(696, 15)

# When I check the encodings below with that in notebook 1, it is to be correct

In [17]:
train_check

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4

In [18]:
print(best_model)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=16, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2400, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)


In [19]:
import pickle


In [20]:
filename = 'output_datasets/models/rfc_model.sav'
pickle.dump(best_model, open(filename, 'wb'))


In [21]:
my_dict = {}
numbers = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16]
sites = ['FH', 'ER', 'WW', 'TC', 'CS', 'KQ', 'AR', 'SL', 'FG', 'WB', 'PF', 'WH',
       'SQ', 'WN', 'BH', 'PH', 'LB']

In [22]:
for i in numbers:
    my_dict[i] = sites[i]

In [23]:
my_dict

{0: 'FH',
 1: 'ER',
 2: 'WW',
 3: 'TC',
 4: 'CS',
 5: 'KQ',
 6: 'AR',
 7: 'SL',
 8: 'FG',
 9: 'WB',
 10: 'PF',
 11: 'WH',
 12: 'SQ',
 13: 'WN',
 14: 'BH',
 15: 'PH',
 16: 'LB'}

In [24]:
print(train_data_formodel[target].unique())
print(uniques)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
Index(['FH', 'ER', 'WW', 'TC', 'CS', 'KQ', 'AR', 'SL', 'FG', 'WB', 'PF', 'WH',
       'SQ', 'WN', 'BH', 'PH', 'LB'],
      dtype='object')


In [25]:
identifiers =  X_test_labeled_df['Analysis']

In [26]:
X_test_labeled_df.head()

,Analysis,Zr90,Ba137,Sr88,Ge72,Cr52,S33,U238,Al27,B11,Mg24,Nd146,Sc45,K39,Pr141,Li7,inlierLabel
0,005_AB_1,0.24,1.06,3.563843,0.80,6.31,487.95,0.210000,306.53,58.850000,25.138079,0.17,1.00,217.05,0.04,19.35,-1
1,006_AB_1,0.30,0.58,1.180000,0.64,4.78,486.15,0.510000,413.97,85.030000,41.100000,0.17,1.25,353.39,0.06,2.61,-1
2,007_AB_1,0.25,0.49,1.250000,0.67,4.62,480.47,0.242569,364.50,80.040000,44.560000,0.24,1.12,329.78,0.07,2.29,1
3,008_AB_2,0.24,4.74,4.350000,0.71,4.46,482.84,0.030000,139.26,70.736204,10.690000,0.63,1.46,90.89,0.20,6.86,-1
4,009_AB_2,0.30,4.22,3.850000,0.98,4.65,506.40,0.060000,180.19,70.736204,18.670000,0.57,1.47,122.66,0.18,8.45,1


In [27]:
results = process_results(best_model, X_test_labeled_df, best_feats, uniques, identifiers)

C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


Pandas Apply:   0%|          | 0/432 [00:00<?, ?it/s]

# test

In [28]:
y_pred = best_model.predict(np.array(X_test_labeled_df[best_feats]))

C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


In [29]:
X_test_labeled_df

,Analysis,Zr90,Ba137,Sr88,Ge72,Cr52,S33,U238,Al27,B11,Mg24,Nd146,Sc45,K39,Pr141,Li7,inlierLabel
0,005_AB_1,0.240000,1.060000,3.563843,0.800000,6.31000,487.950000,0.210000,306.530000,58.850000,25.138079,0.170000,1.000000,217.050000,0.040000,19.350000,-1
1,006_AB_1,0.300000,0.580000,1.180000,0.640000,4.78000,486.150000,0.510000,413.970000,85.030000,41.100000,0.170000,1.250000,353.390000,0.060000,2.610000,-1
2,007_AB_1,0.250000,0.490000,1.250000,0.670000,4.62000,480.470000,0.242569,364.500000,80.040000,44.560000,0.240000,1.120000,329.780000,0.070000,2.290000,1
3,008_AB_2,0.240000,4.740000,4.350000,0.710000,4.46000,482.840000,0.030000,139.260000,70.736204,10.690000,0.630000,1.460000,90.890000,0.200000,6.860000,-1
4,009_AB_2,0.300000,4.220000,3.850000,0.980000,4.65000,506.400000,0.060000,180.190000,70.736204,18.670000,0.570000,1.470000,122.660000,0.180000,8.450000,1
5,010_AB_2,0.300000,3.230000,1.360000,1.340000,4.91000,548.890000,0.170000,479.070000,70.670000,42.990000,0.110000,1.460000,325.750000,0.020000,12.130000,-1
6,011_AB_3,1.552986,0.810000,2.780000,0.450000,11.15000,458.680000,0.110000,661.890000,60.680000,25.138079,0.360000,0.840000,405.460000,0.080000,19.430000,-1
7,012_AB_3,0.240000,1.660000,1.550000,0.680000,6.10000,677.740000,0.100000,232.590000,71.980000,35.130000,0.080000,0.480000,220.470000,0.030000,6.570000,-1
8,013_AB_3,0.230000,0.750000,2.110000,0.410000,5.87000,614.630000,0.050000,336.120000,70.736204,32.320000,0.200000,0.630000,218.880000,0.050000,19.970000,-1
9,014_AB_4,0.670000,27.980000,2.450000,0.480000,6.42000,548.300000,0.230000,429.580000,81.760000,48.350000,0.160000,0.755787,271.980000,0.060000,5.350000,-1


In [30]:
y_pred

array([ 9,  9,  9,  2,  2,  8,  8, 15, 15,  9, 15, 15,  1,  2, 15, 15,  9,
       15,  9,  9,  9,  9,  9, 15, 15, 15, 15, 15, 15, 15, 15, 15, 14, 15,
       15, 15, 15, 15, 15, 15,  4, 15, 15, 15, 15, 10,  6, 15,  6, 16, 10,
       10, 10, 15, 14,  6, 16,  6,  6, 15, 14,  6, 10, 15, 16, 16, 16, 15,
       15,  8,  7, 15, 15,  5,  5,  5,  8,  8,  8, 15,  9,  9, 10,  5,  8,
       15,  8,  5, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 15, 15, 16, 15, 14, 10, 15,  8, 10,  8,  5,  5,  5,  8,
       10,  5, 10, 15, 15,  6, 16, 10,  8, 15,  3, 16, 15, 10, 10,  1, 15,
        1, 10, 10, 15, 10, 10, 10, 10,  8,  8, 15, 15,  2,  2,  2,  2,  1,
        9,  3,  3,  3,  3, 16,  2,  3,  1,  8,  2,  1,  3,  3, 15,  1,  1,
        2,  7,  7,  3,  2,  1,  2,  1,  1,  1,  3,  3,  3,  3,  3,  3,  3,
       16,  3,  3,  3,  3,  3, 10, 14, 15, 15, 15,  2,  2,  2,  3,  2,  9,
        9,  9,  9,  2,  2,  2,  2,  2,  2,  2,  2,  2,  1,  2,  1,  5,  7,
        7,  2,  1,  1,  2

In [31]:


y_pred = best_model.predict(np.array(X_test_labeled_df[best_feats]))
y_pred_proba = best_model.predict_proba(np.array(X_test_labeled_df[best_feats]))

probabilities_df = pd.DataFrame(data = y_pred_proba, columns = uniques)
probabilities_df_final = pd.concat([probabilities_df, identifiers.reset_index(drop = True)], axis = 1)

final_pred_df = pd.concat([pd.Series(y_pred), probabilities_df_final], axis = 1).rename(columns={0:'class_number'})

final_predictions_df = pd.concat([final_pred_df, X_test_labeled_df['inlierLabel']], axis = 1)

uniques_list = list(uniques)
def get_pred_names(row):
    return(uniques_list[row['class_number']])

final_predictions_df['class_predictions'] = final_predictions_df.apply(get_pred_names, axis = 1)

def outlierAssigner(row):
    if row['inlierLabel'] == -1:
        return('other')
    else:
        return(row['class_predictions'])

final_predictions_df['class_predictions'] = final_predictions_df.swifter.apply(outlierAssigner, axis = 1)

C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


Pandas Apply:   0%|          | 0/432 [00:00<?, ?it/s]

# test

In [38]:
results

,class_number,FH,ER,WW,TC,CS,KQ,AR,SL,FG,WB,PF,WH,SQ,WN,BH,PH,LB,Analysis,inlierLabel,class_predictions
0,9,0.015417,0.012917,0.050833,0.000833,0.003750,0.093333,0.001667,0.111250,0.127917,0.247083,0.000833,0.068333,0.022083,0.028333,0.030417,0.178333,0.006667,005_AB_1,-1,other
1,9,0.020000,0.020417,0.022500,0.037917,0.012917,0.057083,0.000000,0.065000,0.010833,0.447917,0.000000,0.032500,0.011250,0.004167,0.052083,0.191667,0.013750,006_AB_1,-1,other
2,9,0.031250,0.021667,0.029583,0.044167,0.002083,0.040417,0.000000,0.025833,0.025417,0.431667,0.000000,0.063333,0.021250,0.012917,0.007500,0.236250,0.006667,007_AB_1,1,WB
3,2,0.059167,0.051250,0.252083,0.027083,0.005000,0.076250,0.000417,0.065000,0.087917,0.199167,0.000417,0.029167,0.015000,0.032917,0.000417,0.092083,0.006667,008_AB_2,-1,other
4,2,0.063333,0.060833,0.266250,0.014167,0.003333,0.095417,0.002500,0.037083,0.128750,0.138750,0.001250,0.037917,0.012917,0.042083,0.001667,0.087500,0.006250,009_AB_2,1,WW
5,8,0.054167,0.060417,0.082500,0.007917,0.000417,0.148750,0.002500,0.018333,0.193750,0.154167,0.002083,0.018333,0.030833,0.022917,0.059167,0.130000,0.013750,010_AB_2,-1,other
6,8,0.113750,0.018750,0.078333,0.002917,0.015417,0.045417,0.024167,0.172500,0.235000,0.052083,0.017500,0.008750,0.020417,0.033333,0.057500,0.074167,0.030000,011_AB_3,-1,other
7,15,0.013750,0.017500,0.112917,0.003750,0.005417,0.008750,0.001250,0.071250,0.033750,0.137917,0.003333,0.051667,0.037917,0.097917,0.040417,0.357083,0.005417,012_AB_3,-1,other
8,15,0.024583,0.020833,0.121667,0.002500,0.003333,0.017083,0.000833,0.094583,0.059167,0.195000,0.002083,0.052917,0.031250,0.101250,0.038750,0.232500,0.001667,013_AB_3,-1,other
9,9,0.029167,0.044583,0.063750,0.003333,0.006250,0.074167,0.001667,0.083333,0.098333,0.245833,0.005000,0.055833,0.015000,0.006250,0.016250,0.235417,0.015833,014_AB_4,-1,other


### I think if there is a problem here it's most likely to be in the encodings of the class, perhaps they are not correct?

In [39]:
results['class_number'].value_counts()

3     99
15    91
2     55
10    51
5     24
1     23
9     22
8     16
16    14
7     11
6      8
11     8
14     5
4      4
13     1
Name: class_number, dtype: int64

In [40]:

#filter for each class prediction, for each class prediction average the probabilities then plot as bar plot, check dists, then filter for outlier or not outlier and bar chart the average probabilities again

### predictions are stored as a variable into memory

In [41]:
%store results

Stored 'results' (DataFrame)


### predictions are outputted as csv file

In [51]:
print(results.head())
#if save_predictions:


results.to_csv('predictions.csv')


   class_number        FH        ER        WW        TC        CS        KQ  \
0             9  0.015417  0.012917  0.050833  0.000833  0.003750  0.093333   
1             9  0.020000  0.020417  0.022500  0.037917  0.012917  0.057083   
2             9  0.031250  0.021667  0.029583  0.044167  0.002083  0.040417   
3             2  0.059167  0.051250  0.252083  0.027083  0.005000  0.076250   
4             2  0.063333  0.060833  0.266250  0.014167  0.003333  0.095417   

         AR        SL        FG        WB        PF        WH        SQ  \
0  0.001667  0.111250  0.127917  0.247083  0.000833  0.068333  0.022083   
1  0.000000  0.065000  0.010833  0.447917  0.000000  0.032500  0.011250   
2  0.000000  0.025833  0.025417  0.431667  0.000000  0.063333  0.021250   
3  0.000417  0.065000  0.087917  0.199167  0.000417  0.029167  0.015000   
4  0.002500  0.037083  0.128750  0.138750  0.001250  0.037917  0.012917   

         WN        BH        PH        LB  Analysis  inlierLabel  \
0  0.0

In [49]:
results.class_predictions.value_counts()

other    131
TC        87
PH        48
WW        36
PF        33
KQ        22
ER        17
WB        14
FG        14
SL         9
LB         8
AR         5
WH         4
BH         2
WN         1
CS         1
Name: class_predictions, dtype: int64